In [1]:
import os
from pymongo import MongoClient
import pandas as pd
import json
import requests
from dotenv import load_dotenv
import re


/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
startmongo = 'service mongod start'
command = 'mongoimport --db companies --collection companies --file Database/companies.json'
def initDB():
    os.popen(startmongo)
os.popen(startmongo)
os.popen('service mongod status')
os.popen(command)

In [3]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

In [4]:
db,coll = (list(connectCollection('companies','companies')))
rest = (list(coll.find({'$and':[{'$or':[{"offices.latitude":{'$ne':None}},{"offices.longitude":{'$ne':None}}]},{'deadpooled_year':None}]})))
value =  rest[0].get('total_money_raised')
value


'$0'

In [5]:
#q = {'$or':[{"offices.latitude":{'$ne':'null'}},{"offices.longitude":{'$ne':'null'}}]}
rest = (list(coll.find({'$and':[{'$or':[{"offices.latitude":{'$ne':None}},{"offices.longitude":{'$ne':None}}]},{'deadpooled_year':None}]})))
clean_db = []
for i in range(len(rest)):
    value = rest[i].get('total_money_raised') 
    valueM = re.findall(r'\d+[?\.\d]?\d',value)
    if valueM == []:
        valueM = [0]
    diction ={
        'name':rest[i]['name'],
        'category':rest[i]['category_code'],
        #'deadpooled':rest[i]['deadpooled_year'],
        'value': {
            'value':float(valueM[0]),
            'qty':value[-1]
        }
        
    }
    for ind in range(len(rest[i]['offices'])):
        if rest[i]['offices']:
            diction[f'Location-{ind+1}'] = {
                'type':'Point',
                'coordinates':[float(rest[i]['offices'][ind]['longitude']), float(rest[i]['offices'][ind]['latitude'])]
            }
        
    
    clean_db.append(diction)
clean_db


[{'name': 'Postini',
  'category': 'web',
  'value': {'value': 0.0, 'qty': '0'},
  'Location-1': {'type': 'Point', 'coordinates': [-122.247573, 37.506885]}},
 {'name': 'Digg',
  'category': 'news',
  'value': {'value': 45.0, 'qty': 'M'},
  'Location-1': {'type': 'Point', 'coordinates': [-122.394523, 37.764726]}},
 {'name': 'Geni',
  'category': 'web',
  'value': {'value': 16.5, 'qty': 'M'},
  'Location-1': {'type': 'Point', 'coordinates': [-118.393064, 34.090368]}},
 {'name': 'Flektor',
  'category': 'games_video',
  'value': {'value': 0.0, 'qty': '0'},
  'Location-1': {'type': 'Point', 'coordinates': [-118.379768, 34.025958]}},
 {'name': 'Fox Interactive Media',
  'category': 'web',
  'value': {'value': 0.0, 'qty': '0'},
  'Location-1': {'type': 'Point', 'coordinates': [-118.39417, 34.076179]}},
 {'name': 'Gizmoz',
  'category': 'web',
  'value': {'value': 18.1, 'qty': 'M'},
  'Location-1': {'type': 'Point', 'coordinates': [-122.169472, 37.48413]}},
 {'name': 'Scribd',
  'category': '

In [6]:
with open('Database/database_clean.json', 'w') as fp:
    json.dump(clean_db, fp)

In [7]:
#command = 'mongoimport --db companies --collection companies_clean --jsonArray --file database_clean.json'
#os.popen(command)

In [8]:
db,coll_clean = (list(connectCollection('companies','companies_clean')))

In [59]:
load_dotenv()
def requestfoursquare(query,lon,lat,radius):
    '''
    lon => float
    lat => float
    radius => meters
    '''
    url = 'https://api.foursquare.com/v2/venues/explore'
    #client = os.getenv("CLIENT_ID")
    #passw = os.getenv("CLIENT_SECRET")
    params = dict(
        client_id=os.getenv("CLIENT_ID"),
        client_secret=os.getenv("CLIENT_SECRET"),
        v='20180335',
        ll='{},{}'.format(lat,lon),
        query=query,
        limit=200,
        radius = radius
    )
    resp = requests.get(url=url, params=params)
    data = json.loads(resp.text)
    return data

In [37]:
def getlocation(data):
    locationslist = []
    for i in range(len(data['response']['groups'][0]['items'])):
        latitude = float((data['response']['groups'][0]['items'][i]['venue'].get('location')).get('lat'))
        longitude = float((data['response']['groups'][0]['items'][i]['venue'].get('location')).get('lng'))
        dictio = {
            'name':(data['response']['groups'][0]['items'][i]['venue']).get('name'),
            'loc':{'type':'Point','coordinates':[longitude,latitude]}            
            }
        locationslist.append(dictio)
    return locationslist


In [60]:
for i in range(len(clean_db)):
    print(i)
    for ind in range(1,(len(clean_db[i])-2)):
        print(i)
        print('enterprise{}, location{}'.format(i,ind))
        lat = clean_db[i]['Location-{}'.format(ind)]['coordinates'][0]
        lon = clean_db[i]['Location-{}'.format(ind)]['coordinates'][1]
        data = requestfoursquare('starbucks',lat,lon,500)
        geo = getlocation(data)
        print(len(geo))

0
0
enterprise0, location1
49
1
1
enterprise1, location1
50
2
2
enterprise2, location1
48
3
3
enterprise3, location1
49
4
4
enterprise4, location1
48
5
5
enterprise5, location1


KeyboardInterrupt: 

In [148]:
clean_db[2]

{'name': 'Geni',
 'category': 'web',
 'value': {'value': 16.5, 'qty': 'M'},
 'Location-1': {'type': 'Point', 'coordinates': [-118.393064, 34.090368]}}